In [2]:
import numpy as np
import pandas as pd
import nltk
from bs4 import BeautifulSoup
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3

In [3]:
data= pd.read_csv("C://Users/Saba Naseem/Desktop/Machine Learning/Codes/kaggle/Document Clustering/movies.csv")

In [4]:
data.columns

Index(['title', 'link', 'syn_link', 'genres', 'plot_synopsis'], dtype='object')

In [8]:
title, synopsis, genre= data['title'], data['plot_synopsis'], data['genres']

In [13]:
synopsis[0][:200]

'In 1947, Andy Dufresne (Tim Robbins), a banker in Maine, is convicted of murdering his wife and her lover, a golf pro. Since the state of Maine has no death penalty, he is given two consecutive life s'

#### Stopwords, stemming, and tokenizing

In [91]:
stopwords= nltk.corpus.stopwords.words("english")
stopwords[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [116]:
def tokenizing(text):
    
    #breaking each word and making them tokens
    tokens=[word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    
    #storing only alpha tokens
    filtered_tokens=[]
    for token in tokens:
        if (re.search('[a-zA-Z]', token)):
            filtered_tokens.append(token)

    return filtered_tokens


from nltk.stem.snowball import SnowballStemmer
stemmer= SnowballStemmer("english")

def stemming(text):
    
    stems =[stemmer.stem(t) for t in text]
    return stems

def token_stems(text):
        
    tokenized_only_vocab=tokenizing(text) 
    #tokens=[word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    stemmed_vocab=stemming(tokenized_only_vocab)
    #storing only alpha tokens
    #filtered_tokens=[]
   # for token in tokens:
    #    if (re.search('[a-zA-Z]', token)):
     #       filtered_tokens.append(token)
            
    #stems =[stemmer.stem(t) for t in filtered_tokens]
    
    return stemmed_vocab       

In [117]:
a= token_stems(synopsis[0])
a[:10]

['in', 'andi', 'dufresn', 'tim', 'robbin', 'a', 'banker', 'in', 'main', 'is']

In [74]:
tokenized_only_vocab=[ tokenizing(word) for word in synopsis]
stemmed_vocab=[ stemming(token) for token in tokenized_only_vocab]

In [86]:
len(stemmed_vocab), len(tokenized_only_vocab)

(100, 100)

#### TF-IDF vectorization

In [97]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf= TfidfVectorizer(max_df=0.8, max_features=200000, min_df=0.2, stop_words='english',
                      use_idf=True, tokenizer=token_stems, ngram_range=(1,3))

%time tfidf_matriz= tfidf.fit_transform(synopsis)

print(tfidf_matriz.shape)

terms=tfidf.get_feature_names()

In [109]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matriz)